In [ ]:
import pandas as pd

In [ ]:
import os

base_path = '/kaggle/input'


folders = [f.path for f in os.scandir(base_path) if f.is_dir()]


print("Folders in Kaggle input:")
for folder in folders:
    print(folder)

print(f"\nTotal number of folders: {len(folders)}")


In [ ]:
df = pd.read_csv('/kaggle/input/moodtheme-1/raw_30s - raw_30s.tsv', delimiter = '\t')
df.head()


In [ ]:
filtered_tags = df.iloc[:,5:]

In [ ]:
contains_instrument = filtered_tags.apply(lambda x: x.astype(str).str.contains('instrument---', na=False)).any(axis=1)
contains_mood = filtered_tags.apply(lambda x: x.astype(str).str.contains('mood/theme---', na=False)).any(axis=1)


mask = contains_instrument & contains_mood


filtered_df = df[mask]

print("Filtered DataFrame:")
print(filtered_df)

In [ ]:
filtered_df

In [ ]:
print("Original data size:", df.shape)
print("Filtered data shape:", filtered_df.shape)

In [ ]:
instrument_mood_tracks = pd.Series(filtered_df['TRACK_ID']).to_list()
print(len(instrument_mood_tracks))

In [ ]:
contains_novoice = ~filtered_df.apply(lambda x: x.astype(str).str.contains('instrument---voice', na=False)).any(axis=1)
novoice_inst_mood = filtered_df[contains_novoice]
print(novoice_inst_mood)

In [ ]:
filtered_df.iloc[0].tolist()

In [ ]:
novoice_list = novoice_inst_mood['PATH'].tolist()
output = [item.split('/')[0] for item in novoice_list]
print("Actual length:", len(output))
print("Unique:", set(output))
print("Unique length:", len(set(output)))

In [ ]:
unique_output = sorted(list(set(output)), key=int)
print(unique_output)

### Create a dictionary that shows #audio files in each tar archive

In [ ]:
tar_raw = {key:[] for key in unique_output}
for t in novoice_list:
    tarfile = t.split('/')[0]
    raw_audio = t.split('/')[1]
    tar_raw[tarfile].append(raw_audio)

#print(tar_raw)

In [ ]:
print(tar_raw['00'])

In [ ]:
num_raws_per_tar = {k:len(v) for k,v in tar_raw.items()}
print(num_raws_per_tar)

In [ ]:
import os
import shutil


dataset_dir = "/kaggle/input/moodtheme-1"
output_dir = "/kaggle/working/filtered_files"


os.makedirs(output_dir, exist_ok=True)

# List of .tar archives
tar_files = [file for file in os.listdir(dataset_dir) if not file.endswith('.tsv')]
print(tar_files)


In [ ]:
id = [t.split('-')[2] for t in tar_files]
print(id)

In [ ]:
'abc.low.mp3'.replace("low.","")

In [ ]:
tarfile_dir = '/raw_30s_audio-low-'
for subfolder, raw_files in tar_raw.items():
    actual_path = dataset_dir+tarfile_dir+subfolder
    subfolder_path = os.path.join(actual_path, subfolder)
    print(subfolder_path)
    
    
    if os.path.exists(subfolder_path):
        print(f"Processing subfolder: {subfolder} from {subfolder_path}")
        
        for file_name in os.listdir(subfolder_path):
            orig_file_name = file_name.replace('low.','')
            print(orig_file_name, file_name)
            
            if orig_file_name in raw_files:
                file_path = os.path.join(subfolder_path, file_name)
                destination_path = os.path.join(output_dir, file_name)
                
                
                shutil.copy(file_path, destination_path)
                print(f"Copied {orig_file_name} to {output_dir}")
    else:
        print(f"Subfolder {subfolder} does not exist in {dataset_dir}")


In [ ]:
shutil.make_archive(output_dir, 'zip', output_dir)